In [ ]:
# pip install beautifulsoup4

# Scrape a list of ingredients from the Harry Potter Wiki

import requests
from bs4 import BeautifulSoup

def removeCitations(tag):
    for sup in tag.find_all('sup'):
            sup.extract()


def extractList(html):
    tab_elements = html.find('div', class_='mw-parser-output').find_all('ul', recursive=False )

    listItems=[]
    for tab in tab_elements:
        items=tab.css.select("li")
        for item in items:
            removeCitations(item)
            link = item.find('a')
            if( link ):
                description = item.get_text().replace(link.text, '').replace('-', '').strip()
                listItems.append({'name':link.text, 'link': link["href"].replace(baseUrl, ''), 'description':  description if len(description)>0 else '' })

    return listItems

baseUrl = "https://harrypotter.fandom.com"
ingredientsUrl = "/wiki/List_of_potion_ingredients"

page = requests.get(baseUrl+ingredientsUrl)
soup = BeautifulSoup(page.content, "html.parser")

ingredients=extractList(soup)

print("-- Ingredients --")
for x in range(len(ingredients)):
    print(ingredients[x])


In [ ]:
# grab ingredient details
for item in ingredients:
    print(item['name'])
    page = requests.get(baseUrl+item['link'])
    html = BeautifulSoup(page.content, "html.parser")

    infoItems = html.find_all('div', class_='pi-item')

    for info in infoItems:
        removeCitations(info)
        propertyName=info['data-source']
        propertyValue=info.css.select_one('div.pi-data-value')

        valueList = propertyValue.css.select_one('ul')
        if( valueList ):
            values=[]
            for valueItem in valueList.find_all('li'):
                values.append(valueItem.get_text())
            item[propertyName]=values    
        else:
            item[propertyName]=propertyValue.get_text()

    content = html.find('div', class_='mw-parser-output')
    
    fullDescription = ''
    for elem in content.find_all(recursive=False):
        if( elem.name == 'p' or elem.name == 'b' or elem.name == 'a' ):
            if( elem.find().name == 'aside'): 
                continue

            removeCitations(elem)
            fullDescription += ' ' + elem.get_text().replace('\r', '').replace('\n',' ').strip()
        if( elem.name == 'h2' ):
            break
       
    descriptionHeader = html.find(id='Description')
    if( descriptionHeader ):
        for elem in descriptionHeader.find_all_next(recursive=False):
            if( elem.name == 'h2' ):
                break
            if( elem.name == 'p' ):
                removeCitations(elem)
                fullDescription += ' ' + elem.get_text().replace('\r', '').replace('\n',' ').strip()
    
    item['fullDescription'] = fullDescription.strip()



import json

SAVE_PATH = "data/ingredients.json"
with open(SAVE_PATH, 'w') as f:
    json.dump(ingredients, f)
